In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
df=pd.read_csv('/content/Laptop_Train_v2 (2).csv')

In [ ]:
df.shape

(2358, 6)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2358 entries, 0 to 2357
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           2358 non-null   int64 
 1   Sentence     2358 non-null   object
 2   Aspect Term  2358 non-null   object
 3   polarity     2358 non-null   object
 4   from         2358 non-null   int64 
 5   to           2358 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 110.7+ KB


In [ ]:
df.columns

Index(['id', 'Sentence', 'Aspect Term', 'polarity', 'from', 'to'], dtype='object')

In [ ]:
df['polarity'].value_counts()

positive    987
negative    866
neutral     460
conflict     45
Name: polarity, dtype: int64

In [ ]:


# Drop rows that contain a specific word in a specific column
word_to_drop = 'conflict'
column_name = 'polarity'
df = df[~df['polarity'].str.contains('conflict')]

# Write the updated DataFrame to a new CSV file
# df.to_csv('file_without_specific_word.csv', index=False)


In [ ]:
df['polarity'].value_counts()

positive    987
negative    866
neutral     460
Name: polarity, dtype: int64

In [ ]:
df = df.drop(['id','from','to'], axis = 1)

In [ ]:
df.head()

,Sentence,Aspect Term,polarity
0,I charge it at night and skip taking the cord ...,cord,neutral
1,I charge it at night and skip taking the cord ...,battery life,positive
2,The tech guy then said the service center does...,service center,negative
3,The tech guy then said the service center does...,"""sales"" team",negative
4,The tech guy then said the service center does...,tech guy,neutral


In [ ]:
df.isnull().sum()

Sentence       0
Aspect Term    0
polarity       0
dtype: int64

In [ ]:
df=df.dropna()

In [ ]:
df['polarity'].value_counts()

positive    987
negative    866
neutral     460
Name: polarity, dtype: int64

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def preprocess_text(Sentence):
    # Remove special characters and digits
    Sentence = re.sub('[^a-zA-Z]', ' ', Sentence)
    # Convert to lowercase
    Sentence = Sentence.lower()
    # Tokenize the text
    tokens = word_tokenize(Sentence)
    # Remove stopwords
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    # Join the tokens back into a string
    Sentence = ' '.join(tokens)
    return Sentence

In [ ]:
Sentence= df['Sentence'].apply(preprocess_text)

In [ ]:
# corpus = []
# for i in range(0, len(df)):
#     review = re.sub('[^a-zA-Z]', ' ',df['Sentence'][i])
#     review = review.lower()
#     review = review.split()
    
#     review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
#     review = ' '.join(review)
#     corpus.append(review)

In [ ]:
Sentence

0         charge night skip taking cord good battery life
1         charge night skip taking cord good battery life
2       tech guy said service center exchange direct c...
3       tech guy said service center exchange direct c...
4       tech guy said service center exchange direct c...
                              ...                        
2352    also use paralles run virtual machines windows...
2353    also use paralles run virtual machines windows...
2354    also use paralles run virtual machines windows...
2356    toshiba handles repair seems vary folks indica...
2357    would like use different operating system alto...
Name: Sentence, Length: 2313, dtype: object

In [ ]:
Sentence[100]

'oh yeah forget expensive shipping hp'

In [ ]:
# ### Dataset Preprocessing
# from nltk.stem.porter import PorterStemmer ##stemming purpose
# ps = PorterStemmer()
# Aspect = []
# for i in range(0, len(df)):
#   review = re.sub('[^a-zA-Z]', ' ', df['Aspect Term'][i])
#   review = review.lower()
#   review = review.split()
    
#   review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
#   review = ' '.join(review)
#   Aspect.append(review)

  
  

In [ ]:
Aspect=df['Aspect Term'].apply(preprocess_text)

In [ ]:
Aspect

0                            cord
1                    battery life
2                  service center
3                      sales team
4                        tech guy
                  ...            
2352         windows home premium
2353    windows server enterprise
2354    windows server enterprise
2356                       repair
2357             operating system
Name: Aspect Term, Length: 2313, dtype: object

In [ ]:
Aspect[3]

'sales team'

In [ ]:
# labels = torch.tensor(df['polarity'].replace({'positive': 2, 'neutral': 1, 'negative': 0}).tolist())

In [ ]:
max_sequence_length = 100
embedding_dim = 100

In [ ]:
# vocabulary_size = len(set(data['review']))

# # Define the aspect vocabulary size based on the number of unique aspects in the dataset
# aspect_vocabulary_size = len(set(data['aspect']))

# # Define the sentiment vocabulary size based on the number of unique sentiments in the dataset
# sentiment_vocabulary_size = len(set(data['sentiment']))

In [ ]:
vocabulary_size = len(set(Sentence))

# Define the aspect vocabulary size based on the number of unique aspects in the dataset
aspect_vocabulary_size = len(set(Aspect))



In [ ]:
vocabulary_size

1452

In [ ]:
aspect_vocabulary_size

912

In [ ]:
# Define the sentiment vocabulary size based on the number of unique sentiments in the dataset
sentiment_vocabulary_size = len(set(df['polarity']))

In [ ]:
sentiment_vocabulary_size

3

In [ ]:
df['polarity'].info()

<class 'pandas.core.series.Series'>
Int64Index: 2313 entries, 0 to 2357
Series name: polarity
Non-Null Count  Dtype 
--------------  ----- 
2313 non-null   object
dtypes: object(1)
memory usage: 100.7+ KB


In [ ]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
# from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers import LSTM, GRU, SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
# from keras.layers.embeddings import Embedding
# from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping

In [ ]:
from tensorflow.keras.layers import Embedding


In [ ]:
# model = Sequential()
# model.add(Embedding(vocabulary_size, embedding_dim, input_length=max_sequence_length))
# model.add(LSTM(128, return_sequences=True))
# model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(aspect_vocabulary_size, activation='softmax'))
# model.add(Dense(1, activation='softmax'))

In [ ]:
# model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 100, 100)          145200    
                                                                 
 lstm_6 (LSTM)               (None, 100, 128)          117248    
                                                                 
 lstm_7 (LSTM)               (None, 64)                49408     
                                                                 
 dense_6 (Dense)             (None, 912)               59280     
                                                                 
 dense_7 (Dense)             (None, 1)                 913       
                                                                 
Total params: 372,049
Trainable params: 372,049
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



In [ ]:
from keras.preprocessing.text import one_hot


In [ ]:
onehot_repr=[one_hot(words,vocabulary_size)for words in Sentence] 
onehot_repr

[[943, 1275, 592, 61, 604, 862, 1238, 798],
 [943, 1275, 592, 61, 604, 862, 1238, 798],
 [784, 1308, 666, 374, 1169, 1322, 1345, 42, 3, 544, 775, 1134, 961, 563],
 [784, 1308, 666, 374, 1169, 1322, 1345, 42, 3, 544, 775, 1134, 961, 563],
 [784, 1308, 666, 374, 1169, 1322, 1345, 42, 3, 544, 775, 1134, 961, 563],
 [1371,
  1258,
  1125,
  455,
  224,
  1427,
  418,
  856,
  1131,
  1393,
  862,
  849,
  827,
  283,
  555,
  1436],
 [1371,
  1258,
  1125,
  455,
  224,
  1427,
  418,
  856,
  1131,
  1393,
  862,
  849,
  827,
  283,
  555,
  1436],
 [1371,
  1258,
  1125,
  455,
  224,
  1427,
  418,
  856,
  1131,
  1393,
  862,
  849,
  827,
  283,
  555,
  1436],
 [1371,
  1258,
  1125,
  455,
  224,
  1427,
  418,
  856,
  1131,
  1393,
  862,
  849,
  827,
  283,
  555,
  1436],
 [827, 86, 623, 1193, 1000],
 [842, 141, 1267, 833, 1160, 307, 935, 515, 1239, 485, 1159, 1395],
 [842, 141, 1267, 833, 1160, 307, 935, 515, 1239, 485, 1159, 1395],
 [842, 141, 1267, 833, 1160, 307, 935, 515

In [ ]:
onehot=[one_hot(words,vocabulary_size)for words in Aspect] 
onehot

[[604],
 [1238, 798],
 [374, 1169],
 [3, 544],
 [784, 1308],
 [1258],
 [455],
 [849],
 [283],
 [86],
 [307],
 [1239],
 [485],
 [1159, 1395],
 [307],
 [455],
 [1140],
 [128, 154],
 [436, 1109, 154],
 [1238],
 [789, 649],
 [426],
 [1217, 1310, 1138],
 [1091, 130],
 [1017, 435],
 [725],
 [869, 175],
 [63],
 [1100],
 [1169, 380],
 [175],
 [790],
 [1202, 801, 1257],
 [220],
 [1080, 427],
 [1140],
 [1186],
 [374],
 [303, 790],
 [391, 175],
 [1120],
 [307],
 [1225, 1320, 1448],
 [420, 374],
 [575],
 [420],
 [307],
 [1096],
 [1293, 374, 906],
 [420],
 [414, 1176],
 [436, 177],
 [1100],
 [1160],
 [1160],
 [1320, 1410],
 [790],
 [480, 545, 942],
 [1167],
 [431],
 [59, 1097],
 [1066],
 [630],
 [1038],
 [232, 420],
 [1258],
 [120],
 [663],
 [1034],
 [1338, 576, 442],
 [283],
 [790],
 [685],
 [875],
 [654, 318],
 [292],
 [1100],
 [166, 630],
 [303, 790],
 [915],
 [369],
 [1236, 442],
 [1038],
 [35, 420],
 [1208],
 [63],
 [1160],
 [517, 389, 876],
 [303, 1153],
 [685],
 [63],
 [1242, 1192],
 [436, 1

In [ ]:
onehot_repr[1]

[943, 1275, 592, 61, 604, 862, 1238, 798]

In [ ]:
onehot[1]

[1238, 798]

In [ ]:
onehot[10]

[307]

In [ ]:
onehot_repr[10]

[842, 141, 1267, 833, 1160, 307, 935, 515, 1239, 485, 1159, 1395]

In [ ]:
onehot_repr[100]


[443, 827, 283, 1257, 175, 1193, 255, 1202, 175]

In [ ]:
onehot[100]

[1202, 175]

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Now you can use pad_sequences function


In [ ]:
# embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
max_sequence_length = 100
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=max_sequence_length)

In [ ]:
onehot_repr[19]

[1203, 1118, 1024, 613, 1363, 1140, 989, 983, 1238, 239]

In [ ]:
embedded_docs[19]

array([1203, 1118, 1024,  613, 1363, 1140,  989,  983, 1238,  239,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0], dtype=int32)

In [ ]:
# import torch

# Now you can use functions and classes from torch library


In [ ]:
# labels = torch.tensor(df['polarity'].replace({'positive': 2, 'neutral': 1, 'negative': 0}).tolist())

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
train_reviews = pad_sequences(embedded_docs, maxlen=max_sequence_length)


In [ ]:
train_reviews

array([[ 943, 1275,  592, ...,    0,    0,    0],
       [ 943, 1275,  592, ...,    0,    0,    0],
       [ 784, 1308,  666, ...,    0,    0,    0],
       ...,
       [1263,  283, 1203, ...,    0,    0,    0],
       [ 232, 1111, 1207, ...,    0,    0,    0],
       [ 544,  515,  283, ...,    0,    0,    0]], dtype=int32)

In [ ]:
test_reviews = pad_sequences(embedded_docs, maxlen=max_sequence_length)

In [ ]:
embedded=pad_sequences(onehot,padding='post',maxlen=1)

In [ ]:
embedded

array([[ 604],
       [ 798],
       [1169],
       ...,
       [1135],
       [1207],
       [ 790]], dtype=int32)

In [ ]:
train_aspect = pad_sequences(embedded, maxlen=1)

In [ ]:
train_aspect

array([[ 604],
       [ 798],
       [1169],
       ...,
       [1135],
       [1207],
       [ 790]], dtype=int32)

In [ ]:
train_aspect[11]

array([1239], dtype=int32)

In [ ]:
test_aspect = pad_sequences(embedded, maxlen=2)

In [ ]:
map_polarity = {'neutral': 2, 'positive': 1, "negative": 0} # map data

train_sentiments = df['polarity'].map(map_polarity)

In [ ]:
map_polarity = {'neutral': 2, 'positive': 1, "negative": 0} # map data

test_sentiments = df['polarity'].map(map_polarity)

In [ ]:
train_sentiments[0]

2

In [ ]:
model = Sequential()
model.add(Embedding(vocabulary_size, embedding_dim, input_length=max_sequence_length))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(aspect_vocabulary_size, activation='softmax'))
model.add(Dense(1, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.fit(train_reviews, train_sentiments, validation_data=(test_reviews, test_sentiments), epochs=50, batch_size=32)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


73/73 [==============================] - 27s 310ms/step - loss: 0.0000e+00 - accuracy: 0.4267 - val_loss: 0.0000e+00 - val_accuracy: 0.4267
Epoch 2/50
73/73 [==============================] - 22s 300ms/step - loss: 0.0000e+00 - accuracy: 0.4267 - val_loss: 0.0000e+00 - val_accuracy: 0.4267
Epoch 3/50
73/73 [==============================] - 22s 300ms/step - loss: 0.0000e+00 - accuracy: 0.4267 - val_loss: 0.0000e+00 - val_accuracy: 0.4267
Epoch 4/50
73/73 [==============================] - 20s 277ms/step - loss: 0.0000e+00 - accuracy: 0.4267 - val_loss: 0.0000e+00 - val_accuracy: 0.4267
Epoch 5/50
73/73 [==============================] - 20s 274ms/step - loss: 0.0000e+00 - accuracy: 0.4267 - val_loss: 0.0000e+00 - val_accuracy: 0.4267
Epoch 6/50
73/73 [==============================] - 20s 274ms/step - loss: 0.0000e+00 - accuracy: 0.4267 - val_loss: 0.0000e+00 - val_accuracy: 0.4267
Epoch 7/50
73/73 [==============================] - 19s 267ms/step - loss: 0.0000e+00 - accuracy: 0.4267 